# LangChain: Models, Prompts and Output Parsers


API calls through LangChain:

- Prompts
- Models
- Output parsers


In [8]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

## Model


In [1]:
from langchain.chat_models import ChatOpenAI
from langchain_fireworks import ChatFireworks

# llm_model = "gpt-4o-mini"
# chat = ChatOpenAI(temperature=0.0, model=llm_model)

llm_model = "accounts/fireworks/models/llama-v3p1-70b-instruct"
chat = ChatFireworks(temperature=0, model=llm_model)

## Prompt template


In [2]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [3]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

In [4]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

In [5]:
customer_style = """American English \
in a calm and respectful tone
"""
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
customer_messages = prompt_template.format_messages(
    style=customer_style, text=customer_email
)

In [7]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>


In [8]:
# Call the LLM to translate to the style of the customer message
customer_response = chat(customer_messages)
print(customer_response.content)

C:\Users\phi.nguyen\AppData\Local\Temp\ipykernel_24392\1158542870.py:2: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  customer_response = chat(customer_messages)


Here is the translated text in a calm and respectful tone, in American English:

"I'm extremely frustrated that my blender lid came off and splattered my kitchen walls with smoothie. To make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I would greatly appreciate your assistance with this issue as soon as possible."


## Output Parsers


Let's start with defining how we would like the LLM output to look like:


In [9]:
{"gift": False, "delivery_days": 5, "price_value": "pretty affordable!"}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [10]:
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [11]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(review_template)
print(prompt_template)

input_variables=['text'] input_types={} partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='For the following text, extract the following information:\n\ngift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'), additional_kwargs={})]


In [13]:
messages = prompt_template.format_messages(text=customer_review)
# chat = ChatOpenAI(temperature=0.0, model=llm_model)
response = chat(messages)
print(response.content)

Here is the extracted information in JSON format:

```
{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```

Let me know if you have any further requests!


In [14]:
type(response.content)

str

## Parse the LLM output string into a Python dictionary


In [15]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [16]:
# Define response_schemas
gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.",
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.",
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.",
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

In [17]:
# Define output parse taking in response_schemas
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased                             as a gift for someone else?                              Answer True if yes,                             False if not or unknown.
	"delivery_days": string  // How many days                                      did it take for the product                                      to arrive? If this                                       information is not found,                                      output -1.
	"price_value": string  // Extract any                                    sentences about the value or                                     price, and output them as a                                     comma separated Python list.
}
```


In [18]:
# Input the format_instruction to the prompt template


messages = prompt_template.format_messages(
    text=customer_review, format_instructions=format_instructions
)
response = chat(messages)

In [19]:
output_dict = output_parser.parse(response.content)
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}